In [1]:
import datasetmaker
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [3]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [4]:
database.columns

Index(['Id', 'Original', 'Eq', 'Frontal 90 Right', 'XCT padded', 'Binary',
       'Material Mask', 'Onlypores', 'UT', 'UT aligned', 'UT aligned amp',
       'UT registered', 'UT lente', 'UT aligned lente', 'UT aligned amp lente',
       'UT registered lente', 'Dataset PatchvsVolfrac 3x3',
       'Dataset PatchvsPacth 3x3', 'Dataset PatchvsVolfrac lente 3x3',
       'Dataset PatchvsPacth lente 3x3', 'Dataset PatchvsVolfrac 5x5',
       'Dataset PatchvsPacth 5x5', 'Dataset PatchvsVolfrac lente 5x5',
       'Dataset PatchvsPacth lente 5x5', 'Dataset PatchvsVolfrac 7x7',
       'Dataset PatchvsPacth 7x7', 'Dataset PatchvsVolfrac lente 7x7',
       'Dataset PatchvsPacth lente 7x7'],
      dtype='object')

In [ ]:
DatasetsFolder = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Airbus\Panel Pegaso')

In [5]:
ids = database['Id']

onlypores_paths = database['Onlypores']

masks = database['Material Mask']

datasets1x3 = database['Dataset PatchvsVolfrac 3x3']

datasets2x3 = database['Dataset PatchvsPacth 3x3']

datasets1x5 = database['Dataset PatchvsVolfrac 5x5']

datasets2x5 = database['Dataset PatchvsPacth 5x5']

datasets1x7 = database['Dataset PatchvsVolfrac 7x7']

datasets2x7 = database['Dataset PatchvsPacth 7x7']

datasets1x3_lente = database['Dataset PatchvsVolfrac lente 3x3']

datasets2x3_lente = database['Dataset PatchvsPacth lente 3x3']

datasets1x5_lente = database['Dataset PatchvsVolfrac lente 5x5']

datasets2x5_lente = database['Dataset PatchvsPacth lente 5x5']

datasets1x7_lente = database['Dataset PatchvsVolfrac lente 7x7']

datasets2x7_lente = database['Dataset PatchvsPacth lente 7x7']

rfs = database['UT registered']

rfs_lente = database['UT registered lente']

In [6]:
for i in range(len(ids)):

    if (onlypores_paths[i] != None) and (masks[i] != None) and (rfs[i] != None):
        print(f"Processing {ids[i]}")
        try:
            #Check if is there a remaining dataset to be processed
            if(datasets1x3[i]==None) or (datasets2x3[i]==None) or (datasets1x5[i]==None) or (datasets2x5[i]==None) or (datasets1x7[i]==None) or (datasets2x7[i]==None):

                print('Loading:',onlypores_paths[i])
                onlypores = read_sequence(onlypores_paths[i])
                print('Loading:',masks[i])
                mask = read_sequence(masks[i])
                print('Loading:',rfs[i])
                rf = tifffile.imread(rfs[i])

                output_folder = DatasetsFolder / str(ids[i])

                if output_folder.exists() == False:
                    output_folder.mkdir()
                
                output_folder = output_folder / 'MonoElement'

                if output_folder.exists() == False:
                    output_folder.mkdir()

                if (datasets1x3[i]==None) or (datasets2x3[i]==None):
                    print('Processing 3x3')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=3)
                    datasets1x3[i] = str(output_folder / 'patch_vs_volfrac_3.csv')
                    datasets2x3[i] = str(output_folder / 'patch_vs_patch_3.csv')
                if (datasets1x5[i]==None) or (datasets2x5[i]==None):
                    print('Processing 5x5')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=5)
                    datasets1x5[i] = str(output_folder / 'patch_vs_volfrac_5.csv')
                    datasets2x5[i] = str(output_folder / 'patch_vs_patch_5.csv')
                if (datasets1x7[i]==None) or (datasets2x7[i]==None):
                    print('Processing 7x7')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=7)
                    datasets1x7[i] = str(output_folder / 'patch_vs_volfrac_7.csv')
                    datasets2x7[i] = str(output_folder / 'patch_vs_patch_7.csv')
            
            else:
                print('All datasets already processed')
        
        except Exception as e:
            print(f"Error processing {ids[i]}")
            print(e)
    
    if (onlypores_paths[i] != None) and (masks[i] != None) and (rfs_lente[i] != None):
        print(f"Processing {ids[i]} lente")
        try:
            #Check if is there a remaining dataset to be processed
            if(datasets1x3_lente[i]==None) or (datasets2x3_lente[i]==None) or (datasets1x5_lente[i]==None) or (datasets2x5_lente[i]==None) or (datasets1x7_lente[i]==None) or (datasets2x7_lente[i]==None):

                print('Loading:',onlypores_paths[i])
                onlypores = read_sequence(onlypores_paths[i])
                print('Loading:',masks[i])
                mask = read_sequence(masks[i])
                print('Loading:',rfs_lente[i])
                rf = tifffile.imread(rfs_lente[i])

                output_folder = DatasetsFolder / str(ids[i])

                if output_folder.exists() == False:
                    output_folder.mkdir()

                output_folder = output_folder / 'MonoElementLens'

                if output_folder.exists() == False:
                    output_folder.mkdir()

                if (datasets1x3_lente[i]==None) or (datasets2x3_lente[i]==None):
                    print('Processing 3x3')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=3)
                    datasets1x3_lente[i] = str(output_folder / 'patch_vs_volfrac_3.csv')
                    datasets2x3_lente[i] = str(output_folder / 'patch_vs_patch_3.csv')
                if (datasets1x5_lente[i]==None) or (datasets2x5_lente[i]==None):
                    print('Processing 5x5')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=5)
                    datasets1x5_lente[i] = str(output_folder / 'patch_vs_volfrac_5.csv')
                    datasets2x5_lente[i] = str(output_folder / 'patch_vs_patch_5.csv')
                if (datasets1x7_lente[i]==None) or (datasets2x7_lente[i]==None):
                    print('Processing 7x7')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=7)
                    datasets1x7_lente[i] = str(output_folder / 'patch_vs_volfrac_7.csv')
                    datasets2x7_lente[i] = str(output_folder / 'patch_vs_patch_7.csv')
            
            else:
                print('All datasets already processed')
        
        except Exception as e:
            print(f"Error processing {ids[i]}")
            print(e)


Processing 1_26 lente
Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\26\segmentation\onlypores


Progress: 100%|██████████| 350/350 [00:41<00:00,  8.46it/s]


Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\26\segmentation\sample_mask


Progress: 100%|██████████| 350/350 [00:41<00:00,  8.34it/s]


Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\probetas\1\26\26_lente_aligned_registered.tif
Processing 3x3
Preprocessing and patching the images...
Patching the images...
2701 2701
Creating the datasets...
Processing 5x5
Preprocessing and patching the images...
Patching the images...
2701 2701
Creating the datasets...
Processing 7x7
Preprocessing and patching the images...
Patching the images...
2701 2701
Creating the datasets...
Processing 1_27
Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\27\segmentation\onlypores


...

Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\probetas\1\13\13_aligned_registered.tif
Processing 3x3
Preprocessing and patching the images...
Patching the images...
2812 2812
Creating the datasets...
Processing 5x5
Preprocessing and patching the images...
Patching the images...
2812 2812
Creating the datasets...
Processing 7x7
Preprocessing and patching the images...
Patching the images...
2812 2812
Creating the datasets...
Processing 1_13 lente
Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\13\segmentation\onlypores


Progress: 100%|██████████| 249/249 [00:37<00:00,  6.65it/s]


Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\13\segmentation\sample_mask


Progress: 100%|██████████| 249/249 [00:38<00:00,  6.50it/s]


Loading: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\probetas\1\13\13_lente_aligned_registered.tif
Processing 3x3
Preprocessing and patching the images...
Patching the images...
2812 2812
Creating the datasets...
Processing 5x5
Preprocessing and patching the images...
Patching the images...
2812 2812
Creating the datasets...
Processing 7x7
Preprocessing and patching the images...
Patching the images...
2812 2812
Creating the datasets...


In [7]:
database['Dataset PatchvsVolfrac 3x3'] = datasets1x3

database['Dataset PatchvsPacth 3x3'] = datasets2x3

database['Dataset PatchvsVolfrac 5x5'] = datasets1x5

database['Dataset PatchvsPacth 5x5'] = datasets2x5

database['Dataset PatchvsVolfrac 7x7'] = datasets1x7

database['Dataset PatchvsPacth 7x7'] = datasets2x7

database['Dataset PatchvsVolfrac lente 3x3'] = datasets1x3_lente

database['Dataset PatchvsPacth lente 3x3'] = datasets2x3_lente

database['Dataset PatchvsVolfrac lente 5x5'] = datasets1x5_lente

database['Dataset PatchvsPacth lente 5x5'] = datasets2x5_lente

database['Dataset PatchvsVolfrac lente 7x7'] = datasets1x7_lente

database['Dataset PatchvsPacth lente 7x7'] = datasets2x7_lente

In [8]:
database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)